In [42]:
import pandas as pd
import re

In [106]:
def read_file():
    column_types = {
            'ID': 'object',  
            'Tweet': 'object',  
            'Replies': 'int64', 
            'Retweets': 'int64',
            'Likes': 'int64', 
            'Bookmarks': 'int64', 
            'Views': 'int64', 
            'Username': 'object',
            'Category': 'object',
        }
    df = pd.read_csv('Tweet_data_scraping.csv', dtype=column_types, parse_dates=['Timestamp', 'PostDate']) #Timestamp และ PostDate เป็น date time
    
    print("df before cleaning:")
    display(df)
    
    return df

In [107]:
def check_data_types(df): 
    df = pd.DataFrame(df)
    
    print("check df types:")
    print(type(df))
    
    print("\ncheck column types:")
    display(df.dtypes)
    
    return df

In [108]:
def is_null(df):
    
    print("check null:")
    null_count = df.isnull().sum()
    print(null_count)
    
    if null_count.sum() > 0:
        print("drop null values")
        df = df.dropna().reset_index(drop=True)
        display(df)
    else:
        print("no null values")

    return df

In [110]:
def is_id_duplicate(df):
    
    print("check duplicate Id:")
    duplicated_rows = df[df['ID'].duplicated(keep=False)]
    display(duplicated_rows)
    
    # ถ้ามี id ซ้ำ เลือกแถวที่มี Timestamp ล่าสุด และ Views มากสุด
    if not duplicated_rows.empty:
        print("drop ID duplicates")
        df = df.sort_values(by=['ID', 'Timestamp', 'Views'], ascending=[True, False, False])
        df = df.drop_duplicates(subset=['ID'], keep='first').reset_index(drop=True)
        display(df)
    else:
        print("no duplicate ID")    
        
    return df

In [111]:
def clean_tweet(df):    
    # คลีนข้อมูลในคอลัมน์ 'Tweet'
    print("clean text:")
    df['Tweet'] = df['Tweet'].str.replace(r'\n', ' ', regex=True)  # แทนที่ \n ด้วยช่องว่าง
    df['Tweet'] = df['Tweet'].str.replace(r'\s+', ' ', regex=True)  # แทนที่หลายช่องว่างด้วยช่องว่างเดียว
    df['Tweet'] = df['Tweet'].str.strip()  # ลบการเว้นวรรคจากขอบ
    display(df)   
    return df

In [137]:
# ลบข้อความที่ซ้ำจาก Tweet
def is_tweet_duplicate(df):
    
    print("check duplicate 'Tweet':")
    unique_duplicates = df[df['Tweet'].duplicated(keep='first') | df['Tweet'].duplicated(keep='last')]
    display(unique_duplicates)
    
    if not unique_duplicates.empty:
        print("drop tweet duplicate")
        df_unique_tweet = df.sort_values(by=['Tweet', 'Timestamp', 'Views'], ascending=[True, False, False])
        df_unique_tweet = df_unique_tweet.drop_duplicates(subset='Tweet', keep='first').reset_index(drop=True) # ถ้า Tweet ซ้ำกัน เลือกข้อมูลที่ Timestamp ล่าสุดและ Viewer มากสุด
        df_unique_tweet = df_unique_tweet.reset_index(drop=True)
        display(df_unique_tweet)
    else:
        print("no duplicate tweet")
        
    return df_unique_tweet

In [115]:
# เพิ่มคอลัมน์ Cleaned_Tweet และ Hashtags
def split_tweet_and_hashtags(df):
    print("add column 'Cleaned_Tweet' and 'Hashtags':")
    # ดึงข้อความทั้งหมดที่ไม่ใช่ Hashtags
    df['Cleaned_Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'#\S+', '', x).strip())
    # ดึงข้อความทั้งหมดที่เป็น Hashtags
    df['Hashtags'] = df['Tweet'].apply(lambda x: re.findall(r'#([\wก-๙]+)', x))
    display(df)
    return df 

In [164]:
# ลบข้อความที่ซ้ำจาก Cleaned_Tweet
def drop_clean_tweet_duplicate(df):
    
    print("check duplicate 'Cleaned_Tweet':")
    unique_duplicates = df[df['Cleaned_Tweet'].duplicated(keep=False)]
    display(unique_duplicates)
    
    if not unique_duplicates.empty:
        print("drop 'Cleaned_Tweet' duplicate")
        df = df.sort_values(by=['Cleaned_Tweet', 'Timestamp', 'Views'], ascending=[True, False, False])
        df = df.drop_duplicates(subset='Cleaned_Tweet', keep='first').reset_index(drop=True)
        df = df.reset_index(drop=True)
        display(df)
    else:
        print("no Cleaned_Tweet duplicate")
        
    return df

In [165]:
def column_order(df):
    print("reorder columns:")
    new_column_order = ["ID", "Tweet",  "Cleaned_Tweet", "Replies", "Retweets", "Likes", "Bookmarks", "Views", "Username", "PostDate", "Timestamp", "Hashtags", "Category"]
    df = df[new_column_order]
    display(df)
    return df

In [172]:
def reduce_whitespace(s):
    return re.sub(r'\s{2,}', ' ', s)

# คัดกรองคำที่ไม่เหมาะสมออก
def is_jobs_by_keyword(title, keywords):
    title = reduce_whitespace(title.lower())
    return any(keyword in title for keyword in keywords)

In [177]:
def save_csv(main_df):
    
    print("df after cleaning:")
    display(main_df)

    file_path = "Tweet_data_cleaning.csv"
    
    column_types = {
            'ID': 'object',  
            'Tweet': 'object',
            'Cleaned_Tweet': 'object',  
            'Replies': 'int64', 
            'Retweets': 'int64',
            'Likes': 'int64', 
            'Bookmarks': 'int64', 
            'Views': 'int64', 
            'Username': 'object',
            'PostDate' : 'datetime64[ns]',
            'Timestamp' : 'datetime64[ns]',
            'Hashtag' : 'object',
            'Category': 'object',
        }
    
    # แปลง Data type
    for column, dtype in column_types.items():
        main_df[column] = main_df[column].astype(dtype)

    # บันทึกข้อมูลลงไฟล์ CSV
    main_df.to_csv(
        file_path,
        index=False,
        encoding="utf-8",
        # dtype=column_types
    )

In [178]:
def data_manipulation():
    df = read_file()
    df_check_types = check_data_types(df)
    df_check_null = is_null(df_check_types)
    df_check_id = is_id_duplicate(df_check_null)
    df_clean_text = clean_tweet(df_check_id)
    df_check_tweet = is_tweet_duplicate(df_clean_text)
    df_add_columns = split_tweet_and_hashtags(df_check_tweet)
    df_check_clean_tweet = drop_clean_tweet_duplicate(df_add_columns)
    df_reorder_columns = column_order(df_check_clean_tweet)
    df_filter = df_reorder_columns[df_reorder_columns['Tweet'].apply(lambda Tweet: not is_jobs_by_keyword(Tweet, ["เย็ด", "เยด", "เสียว", "เซ็กส์",  "เซ็ก", "เซก", "เงี่ยน", "fwb", "FWB", "ons", "ONS", "18+"]))]
    df_result = df_filter.reset_index(drop=True)
    return df_result

Data Manipulation

In [175]:
df = data_manipulation()

df before cleansing:


,ID,Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Category
0,1783681490318143902,รศ. ดร.วีระภาส คุณรัตนสิริ รองคณบดีฝ่ายสารสนเท...,0,0,0,0,296,Grad_KU,2024-04-26 02:16:21,2024-11-29 01:24:19,มหาวิทยาลัยเกษตรศาสตร์
1,1728921798811746633,มาวิ่งงาน Run For Fisheries #RunForFisheries20...,0,0,0,0,193,wissarut106,2023-11-26 23:40:53,2024-11-29 01:24:19,มหาวิทยาลัยเกษตรศาสตร์
2,1657651670560698368,ถ้าเราอยากอยู่ห้องเดียวกับเพื่อนสามารถทำยังไงไ...,1,0,0,0,450,bambysocute,2023-05-14 07:38:50,2024-11-29 01:24:19,มหาวิทยาลัยเกษตรศาสตร์
3,1691777404833009925,แอดไลน์ http://bit.ly/3AVZOt6 แอดไลน์ตอนนี้ ...,0,1,0,0,917,theplannertutor,2023-08-16 11:42:19,2024-11-29 01:24:19,มหาวิทยาลัยเกษตรศาสตร์
4,1660348104267423744,รบกวนช่วยรีหน่อยค่ะ\nเวลาประมาณ 23.40 น. ใครที...,1,38,2,0,2960,DZemb_1991,2023-05-21 18:13:30,2024-11-29 01:24:19,มหาวิทยาลัยเกษตรศาสตร์
...,...,...,...,...,...,...,...,...,...,...,...
407,1674686048193101828,เน็ต ais ไม่อั้นไม่ลดสปีด ปี 2566 #AIS เน็ตAI...,0,0,1,0,278,TANTAWAN014849,2023-06-30 07:47:22,2024-12-24 04:46:36,เน็ตเอไอเอส
408,1736963648411848741,มีโปรย้ายค่ายเบอร์เดิมๆคุ้มๆไหมคะ #ais #Ais #เ...,0,0,3,0,466,kkybeiei,2023-12-19 04:16:19,2024-12-24 04:46:36,เน็ตเอไอเอส
409,1640948608521367552,หลุดดีจังนะ เวลาสำคัญๆต้องส่งงานเนี่ย #เน็ตเอไ...,0,0,0,0,64,Kit78065566,2023-03-29 05:26:50,2024-12-24 04:46:36,เน็ตเอไอเอส
410,1653052012693692419,จ่ายเดือนละตั้งหชายบาทเน็ตฟลิกยังดูไม่ได้ กุงึ...,0,0,0,0,162,alittlePP15,2023-05-01 15:01:26,2024-12-24 04:46:36,เน็ตเอไอเอส


check df types:
<class 'pandas.core.frame.DataFrame'>

check column types:


ID                   object
Tweet                object
Replies               int64
Retweets              int64
Likes                 int64
Bookmarks             int64
Views                 int64
Username             object
PostDate     datetime64[ns]
Timestamp    datetime64[ns]
Category             object
dtype: object

check null:
ID           0
Tweet        0
Replies      0
Retweets     0
Likes        0
Bookmarks    0
Views        0
Username     0
PostDate     0
Timestamp    0
Category     0
dtype: int64
no null values
check duplicate Id:


...

,ID,Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Category,Cleaned_Tweet,Hashtags
0,1851445774359654773,#trueล่ม #เน็ตทรู #True #truevision #trueonlin...,0,1,0,0,3040,luckyseyo,2024-10-30 02:07:25,2024-12-24 04:45:53,เน็ตทรู,,"[trueล่ม, เน็ตทรู, True, truevision, trueonlin..."
1,1834548488585236847,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์...,0,6,22,3,2371,TCASterApp,2024-09-13 11:03:38,2024-12-06 02:22:13,มหาวิทยาลัยเกษตรศาสตร์,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์...,"[TCASter, Mytcas, TCAS68, DEK68, admission, po..."
2,1706623376301896183,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เ...,0,0,1,0,131,thecoverageinfo,2023-09-26 10:54:55,2024-11-29 01:24:19,มหาวิทยาลัยเกษตรศาสตร์,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เ...,"[แพทยศาสตร์, เกษตรศาสตร์, แพทยสภา]"
3,1856644312022618165,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทย...,0,0,0,0,1252,TCASterApp,2024-11-13 10:24:33,2024-12-24 04:44:51,มหาวิทยาลัยเกษตรศาสตร์,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทย...,"[TCASter, Mytcas, TCAS67, TCAS68, DEK68, dek69..."
4,1820651672152285291,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยา...,1,12,31,5,3161,TCASterApp,2024-08-06 02:42:39,2024-12-24 04:43:33,มหาวิทยาลัยเกษตรศาสตร์,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยา...,"[DEK68, TCASter, Mytcas, TCAS67, TCAS68, DEK68..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,1761690508253184465,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ช...,0,0,0,0,995,Kom_chad_luek,2024-02-25 09:52:02,2024-12-24 04:43:33,มหาวิทยาลัยเกษตรศาสตร์,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ช...,"[สุวัจน์, เกษตรศาสตร์, kurun3]"
364,1869368995276841099,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : ...,0,0,0,0,672,BUSshine__id,2024-12-18 13:07:54,2024-12-24 04:44:51,มหาวิทยาลัยเกษตรศาสตร์,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : ...,"[เกษตรศาสตร์, ทีมมก, kasetsart, ku, kuopenhous..."
365,1771364425053896829,"↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS M...",0,0,0,0,198,kjn_kt,2024-03-23 02:32:43,2024-12-03 22:27:43,เน็ตเอไอเอส,"↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS M...","[เน็ตเอไอเอส, nPerf]"
366,1856541134694040023,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภ...,0,0,0,0,491,naaegyo_shop,2024-11-13 03:34:34,2024-12-24 04:44:51,มหาวิทยาลัยเกษตรศาสตร์,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภ...,"[ทีมมก, มก, เกษตรบางเขน, เกษตรศาสตร์, KU, ส่งต..."


reorder columns:


,ID,Tweet,Cleaned_Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Hashtags,Category
0,1851445774359654773,#trueล่ม #เน็ตทรู #True #truevision #trueonlin...,,0,1,0,0,3040,luckyseyo,2024-10-30 02:07:25,2024-12-24 04:45:53,"[trueล่ม, เน็ตทรู, True, truevision, trueonlin...",เน็ตทรู
1,1834548488585236847,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์...,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์...,0,6,22,3,2371,TCASterApp,2024-09-13 11:03:38,2024-12-06 02:22:13,"[TCASter, Mytcas, TCAS68, DEK68, admission, po...",มหาวิทยาลัยเกษตรศาสตร์
2,1706623376301896183,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เ...,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เ...,0,0,1,0,131,thecoverageinfo,2023-09-26 10:54:55,2024-11-29 01:24:19,"[แพทยศาสตร์, เกษตรศาสตร์, แพทยสภา]",มหาวิทยาลัยเกษตรศาสตร์
3,1856644312022618165,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทย...,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทย...,0,0,0,0,1252,TCASterApp,2024-11-13 10:24:33,2024-12-24 04:44:51,"[TCASter, Mytcas, TCAS67, TCAS68, DEK68, dek69...",มหาวิทยาลัยเกษตรศาสตร์
4,1820651672152285291,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยา...,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยา...,1,12,31,5,3161,TCASterApp,2024-08-06 02:42:39,2024-12-24 04:43:33,"[DEK68, TCASter, Mytcas, TCAS67, TCAS68, DEK68...",มหาวิทยาลัยเกษตรศาสตร์
...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,1761690508253184465,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ช...,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ช...,0,0,0,0,995,Kom_chad_luek,2024-02-25 09:52:02,2024-12-24 04:43:33,"[สุวัจน์, เกษตรศาสตร์, kurun3]",มหาวิทยาลัยเกษตรศาสตร์
364,1869368995276841099,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : ...,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : ...,0,0,0,0,672,BUSshine__id,2024-12-18 13:07:54,2024-12-24 04:44:51,"[เกษตรศาสตร์, ทีมมก, kasetsart, ku, kuopenhous...",มหาวิทยาลัยเกษตรศาสตร์
365,1771364425053896829,"↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS M...","↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS M...",0,0,0,0,198,kjn_kt,2024-03-23 02:32:43,2024-12-03 22:27:43,"[เน็ตเอไอเอส, nPerf]",เน็ตเอไอเอส
366,1856541134694040023,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภ...,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภ...,0,0,0,0,491,naaegyo_shop,2024-11-13 03:34:34,2024-12-24 04:44:51,"[ทีมมก, มก, เกษตรบางเขน, เกษตรศาสตร์, KU, ส่งต...",มหาวิทยาลัยเกษตรศาสตร์


df after cleaning:


,ID,Tweet,Cleaned_Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Hashtags,Category
0,1851445774359654773,#trueล่ม #เน็ตทรู #True #truevision #trueonlin...,,0,1,0,0,3040,luckyseyo,2024-10-30 02:07:25,2024-12-24 04:45:53,"[trueล่ม, เน็ตทรู, True, truevision, trueonlin...",เน็ตทรู
1,1834548488585236847,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์...,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์...,0,6,22,3,2371,TCASterApp,2024-09-13 11:03:38,2024-12-06 02:22:13,"[TCASter, Mytcas, TCAS68, DEK68, admission, po...",มหาวิทยาลัยเกษตรศาสตร์
2,1706623376301896183,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เ...,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เ...,0,0,1,0,131,thecoverageinfo,2023-09-26 10:54:55,2024-11-29 01:24:19,"[แพทยศาสตร์, เกษตรศาสตร์, แพทยสภา]",มหาวิทยาลัยเกษตรศาสตร์
3,1856644312022618165,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทย...,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทย...,0,0,0,0,1252,TCASterApp,2024-11-13 10:24:33,2024-12-24 04:44:51,"[TCASter, Mytcas, TCAS67, TCAS68, DEK68, dek69...",มหาวิทยาลัยเกษตรศาสตร์
4,1820651672152285291,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยา...,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยา...,1,12,31,5,3161,TCASterApp,2024-08-06 02:42:39,2024-12-24 04:43:33,"[DEK68, TCASter, Mytcas, TCAS67, TCAS68, DEK68...",มหาวิทยาลัยเกษตรศาสตร์
...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1761690508253184465,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ช...,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ช...,0,0,0,0,995,Kom_chad_luek,2024-02-25 09:52:02,2024-12-24 04:43:33,"[สุวัจน์, เกษตรศาสตร์, kurun3]",มหาวิทยาลัยเกษตรศาสตร์
347,1869368995276841099,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : ...,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : ...,0,0,0,0,672,BUSshine__id,2024-12-18 13:07:54,2024-12-24 04:44:51,"[เกษตรศาสตร์, ทีมมก, kasetsart, ku, kuopenhous...",มหาวิทยาลัยเกษตรศาสตร์
348,1771364425053896829,"↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS M...","↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS M...",0,0,0,0,198,kjn_kt,2024-03-23 02:32:43,2024-12-03 22:27:43,"[เน็ตเอไอเอส, nPerf]",เน็ตเอไอเอส
349,1856541134694040023,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภ...,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภ...,0,0,0,0,491,naaegyo_shop,2024-11-13 03:34:34,2024-12-24 04:44:51,"[ทีมมก, มก, เกษตรบางเขน, เกษตรศาสตร์, KU, ส่งต...",มหาวิทยาลัยเกษตรศาสตร์


Save

In [179]:
save_csv(df)

df after cleaning:


,ID,Tweet,Cleaned_Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Hashtags,Category
0,1851445774359654773,#trueล่ม #เน็ตทรู #True #truevision #trueonlin...,,0,1,0,0,3040,luckyseyo,2024-10-30 02:07:25,2024-12-24 04:45:53,"[trueล่ม, เน็ตทรู, True, truevision, trueonlin...",เน็ตทรู
1,1834548488585236847,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์...,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์...,0,6,22,3,2371,TCASterApp,2024-09-13 11:03:38,2024-12-06 02:22:13,"[TCASter, Mytcas, TCAS68, DEK68, admission, po...",มหาวิทยาลัยเกษตรศาสตร์
2,1706623376301896183,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เ...,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เ...,0,0,1,0,131,thecoverageinfo,2023-09-26 10:54:55,2024-11-29 01:24:19,"[แพทยศาสตร์, เกษตรศาสตร์, แพทยสภา]",มหาวิทยาลัยเกษตรศาสตร์
3,1856644312022618165,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทย...,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทย...,0,0,0,0,1252,TCASterApp,2024-11-13 10:24:33,2024-12-24 04:44:51,"[TCASter, Mytcas, TCAS67, TCAS68, DEK68, dek69...",มหาวิทยาลัยเกษตรศาสตร์
4,1820651672152285291,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยา...,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยา...,1,12,31,5,3161,TCASterApp,2024-08-06 02:42:39,2024-12-24 04:43:33,"[DEK68, TCASter, Mytcas, TCAS67, TCAS68, DEK68...",มหาวิทยาลัยเกษตรศาสตร์
...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1761690508253184465,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ช...,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ช...,0,0,0,0,995,Kom_chad_luek,2024-02-25 09:52:02,2024-12-24 04:43:33,"[สุวัจน์, เกษตรศาสตร์, kurun3]",มหาวิทยาลัยเกษตรศาสตร์
347,1869368995276841099,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : ...,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : ...,0,0,0,0,672,BUSshine__id,2024-12-18 13:07:54,2024-12-24 04:44:51,"[เกษตรศาสตร์, ทีมมก, kasetsart, ku, kuopenhous...",มหาวิทยาลัยเกษตรศาสตร์
348,1771364425053896829,"↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS M...","↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS M...",0,0,0,0,198,kjn_kt,2024-03-23 02:32:43,2024-12-03 22:27:43,"[เน็ตเอไอเอส, nPerf]",เน็ตเอไอเอส
349,1856541134694040023,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภ...,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภ...,0,0,0,0,491,naaegyo_shop,2024-11-13 03:34:34,2024-12-24 04:44:51,"[ทีมมก, มก, เกษตรบางเขน, เกษตรศาสตร์, KU, ส่งต...",มหาวิทยาลัยเกษตรศาสตร์


manual check

In [127]:
df[df.ID=='1850096883609518413']

,ID,Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Category
89,1850096883609518413,อยู่ๆก็ใช้ไม่ได้เลย\nตอนนี้ใช้ ais อยู่ แต่อีก...,0,0,1,0,995,GBDJTJCGPMIj11F,2024-10-26 08:47:24,2024-12-06 02:13:41,เน็ตทรู


In [ ]:
# เช็คข้อมูลที่อยากตรวจสอบ
result = df[df['Username'] == "netais_"]
with pd.option_context('display.max_colwidth', None):
    display(result)

,ID,Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Category,Cleaned_Tweet,Hashtags
271,1680381832918093825,เน็ตais ไม่อั้นไม่ลดสปีด ปี 2566 #net #เน็ตaisซิมใหม่ #เน็ตaisไม่ลดสปีด #เน็ตAIS #เน็ตวันทูคอล #AIS5G #เน็ตไม่อั้นไม่ลดสปีด #เน็ตไม่อั้น #โปรเน็ตais #เน็ตฟลิกซ์ #เน็ตเอไอเอส #เน็ตมือถือ #เน็ตais #ais #Ais #เน็ตไม่ลดสปีด #วันทูคอล #AISFibre #AISPLAY #เน็ตฟลิกซ์ไทย,0,1,4,0,2576,netais_,2023-07-16 01:00:23,2024-12-24 04:46:36,เน็ตเอไอเอส,เน็ตais ไม่อั้นไม่ลดสปีด ปี 2566,"[net, เน็ตaisซิมใหม่, เน็ตaisไม่ลดสปีด, เน็ตAIS, เน็ตวันทูคอล, AIS5G, เน็ตไม่อั้นไม่ลดสปีด, เน็ตไม่อั้น, โปรเน็ตais, เน็ตฟลิกซ์, เน็ตเอไอเอส, เน็ตมือถือ, เน็ตais, ais, Ais, เน็ตไม่ลดสปีด, วันทูคอล, AISFibre, AISPLAY, เน็ตฟลิกซ์ไทย]"
309,1719187471504990566,เน็ตไม่อั้นไม่ลดสปีด ปี 2566 เน็ตเอไอเอส-วันทูคอลยอดนิยม สมัครได้ทุกซิม AIS-12call #net #เน็ตaisซิมใหม่ #เน็ตaisไม่ลดสปีด #เน็ตAIS #เน็ตวันทูคอล #AIS5G #เน็ตไม่อั้นไม่ลดสปีด #เน็ตไม่อั้น #เน็ตบ้าน #เน็ตฟลิกซ์ #เน็ตเอไอเอส #เน็ตมือถือ #เน็ตais #ais #Ais #เน็ตไม่ลดสปีด,0,0,1,0,2151,netais_,2023-10-31 03:00:08,2024-11-29 01:26:32,เน็ตเอไอเอส,เน็ตไม่อั้นไม่ลดสปีด ปี 2566 เน็ตเอไอเอส-วันทูคอลยอดนิยม สมัครได้ทุกซิม AIS-12call,"[net, เน็ตaisซิมใหม่, เน็ตaisไม่ลดสปีด, เน็ตAIS, เน็ตวันทูคอล, AIS5G, เน็ตไม่อั้นไม่ลดสปีด, เน็ตไม่อั้น, เน็ตบ้าน, เน็ตฟลิกซ์, เน็ตเอไอเอส, เน็ตมือถือ, เน็ตais, ais, Ais, เน็ตไม่ลดสปีด]"
337,1686042548593475584,โปรเน็ต 𝐀𝐈𝐒 ไม่ลดสปีด #net #เน็ตaisซิมใหม่ #เน็ตaisไม่ลดสปีด #เน็ตวันทูคอล #AIS5G #เน็ตไม่อั้นไม่ลดสปีด #เน็ตไม่อั้น #เน็ตฟลิกซ์ #เน็ตเอไอเอส #เน็ตมือถือ #เน็ตais #ais #Ais #เน็ตไม่ลดสปีด #วันทูคอล #เน็ตฟลิกซ์ไทย #โปรเน็ตais #สมัครเน็ต #TwitterX #มาตาลดาep16,0,1,2,0,1403,netais_,2023-07-31 15:54:03,2024-12-06 02:14:25,เน็ตเอไอเอส,โปรเน็ต 𝐀𝐈𝐒 ไม่ลดสปีด,"[net, เน็ตaisซิมใหม่, เน็ตaisไม่ลดสปีด, เน็ตวันทูคอล, AIS5G, เน็ตไม่อั้นไม่ลดสปีด, เน็ตไม่อั้น, เน็ตฟลิกซ์, เน็ตเอไอเอส, เน็ตมือถือ, เน็ตais, ais, Ais, เน็ตไม่ลดสปีด, วันทูคอล, เน็ตฟลิกซ์ไทย, โปรเน็ตais, สมัครเน็ต, TwitterX, มาตาลดาep16]"


In [171]:
with pd.option_context('display.max_colwidth', None):
    display(df)

,ID,Tweet,Cleaned_Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Hashtags,Category
0,1851445774359654773,#trueล่ม #เน็ตทรู #True #truevision #trueonline #truemoveH,,0,1,0,0,3040,luckyseyo,2024-10-30 02:07:25,2024-12-24 04:45:53,"[trueล่ม, เน็ตทรู, True, truevision, trueonline, truemoveH]",เน็ตทรู
1,1834548488585236847,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์ วิทยาเขตศรีราชา เริ่มรอบพอร์ต 1 ตุลาคมนี้แล้ว! . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ #TCASter #Mytcas #TCAS68 #DEK68 #admission #portfolio #Quota #DirectAdmission #KU #kasetsart #เกษตรศาสตร์,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์ วิทยาเขตศรีราชา เริ่มรอบพอร์ต 1 ตุลาคมนี้แล้ว! . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ,0,6,22,3,2371,TCASterApp,2024-09-13 11:03:38,2024-12-06 02:22:13,"[TCASter, Mytcas, TCAS68, DEK68, admission, portfolio, Quota, DirectAdmission, KU, kasetsart, เกษตรศาสตร์]",มหาวิทยาลัยเกษตรศาสตร์
2,1706623376301896183,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เกษตรฯ' พร้อมเปิดสอนปี 67 รองรับได้ 48 คน #แพทยศาสตร์ #เกษตรศาสตร์ #แพทยสภา https://thecoverage.info/news/content/5464…,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เกษตรฯ' พร้อมเปิดสอนปี 67 รองรับได้ 48 คน https://thecoverage.info/news/content/5464…,0,0,1,0,131,thecoverageinfo,2023-09-26 10:54:55,2024-11-29 01:24:19,"[แพทยศาสตร์, เกษตรศาสตร์, แพทยสภา]",มหาวิทยาลัยเกษตรศาสตร์
3,1856644312022618165,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทยาลัยอีกครั้งก่อนสมัคร . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ : iOS https://tcaster.info/3P6Lu8h : Android https://tcaster.info/3PdmxYK #TCASter #Mytcas #TCAS67 #TCAS68 #DEK68 #dek69 #KU #เกษตรศาสตร์ #มหาวิทยาลัย #สอบติดมหาลัย,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทยาลัยอีกครั้งก่อนสมัคร . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ : iOS https://tcaster.info/3P6Lu8h : Android https://tcaster.info/3PdmxYK,0,0,0,0,1252,TCASterApp,2024-11-13 10:24:33,2024-12-24 04:44:51,"[TCASter, Mytcas, TCAS67, TCAS68, DEK68, dek69, KU, เกษตรศาสตร์, มหาวิทยาลัย, สอบติดมหาลัย]",มหาวิทยาลัยเกษตรศาสตร์
4,1820651672152285291,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยาลัยเกษตรศาสตร์กันบ้าง พี่ทีเชื่อว่าเป็นมหาลัยในฝันของน้อง ๆ #DEK68 หลายคนเลย . จะมีคณะที่การแข่งขันสูงปรี๊ดบ้าง มาดูกันเลย~ . #TCASter #Mytcas #TCAS67 #TCAS68 #DEK68 #admission #รอบ3 #แอดมิชชัน #เกษตร #เกษตรศาสตร์ #หมาวิทยาลัยเกษตร,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยาลัยเกษตรศาสตร์กันบ้าง พี่ทีเชื่อว่าเป็นมหาลัยในฝันของน้อง ๆ หลายคนเลย . จะมีคณะที่การแข่งขันสูงปรี๊ดบ้าง มาดูกันเลย~ .,1,12,31,5,3161,TCASterApp,2024-08-06 02:42:39,2024-12-24 04:43:33,"[DEK68, TCASter, Mytcas, TCAS67, TCAS68, DEK68, admission, รอบ3, แอดมิชชัน, เกษตร, เกษตรศาสตร์, หมาวิทยาลัยเกษตร]",มหาวิทยาลัยเกษตรศาสตร์
...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1761690508253184465,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ชิงถ้วยพระราชทาน หารายได้สร้างโรงพยาบาลคณะแพทย์ศาสตร์ ม.เกษตรศาสตร์ #สุวัจน์ #เกษตรศาสตร์ #kurun3,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ชิงถ้วยพระราชทาน หารายได้สร้างโรงพยาบาลคณะแพทย์ศาสตร์ ม.เกษตรศาสตร์,0,0,0,0,995,Kom_chad_luek,2024-02-25 09:52:02,2024-12-24 04:43:33,"[สุวัจน์, เกษตรศาสตร์, kurun3]",มหาวิทยาลัยเกษตรศาสตร์
347,1869368995276841099,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : thai.busofficial See u tomorrow https://instagram.com/reel/DDtvzr7BA_B/?igsh=MnM2Nmp6ODF5bzdl… #เกษตรศาสตร์ #ทีมมก #kasetsart #ku #kuopenhouse #tcas #TCAS68 #BUSbecauseofyouishine #THAIchayanon,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : thai.busofficial See u tomorrow https://instagram.com/reel/DDtvzr7BA_B/?igsh=MnM2Nmp6ODF5bzdl…,0,0,0,0,672,BUSshine__id,2024-12-18 13:07:54,2024-12-24 04:44:51,"[เกษตรศาสตร์, ทีมมก, kasetsart, ku, kuopenhouse, tcas, TCAS68, BUSbecauseofyouishine, THAIchayanon]",มหาวิทยาลัยเกษตรศาสตร์
348,1771364425053896829,"↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS Mobile / #เน็ตเอไอเอส / #nPerf v2.13.6 /","↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS Mobile / / v2.13.6 /",0,0,0,0,